In [30]:

from sklearn.metrics import r2_score

# After training the model
y_pred = model.predict(X_test)
y_pred = y_pred if isinstance(y_pred, np.ndarray) else y_pred.numpy()  # Convert to numpy array if necessary

r2 = r2_score(y_test, y_pred)
print(f'R^2 score: {r2}')


In [3]:
tf.config.optimizer.set_jit(True) # Enable XLA.

In [4]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

## Random dataset generated

In [5]:
n_samples = 150000
n_features = 10
X, y = make_friedman1(n_samples=n_samples, n_features=n_features, noise=1.0)
X

## Split dataset into train and test sets

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

In [7]:
train_df = pd.DataFrame(X_train)
train_y_df = pd.DataFrame(y_train, columns=['y'])
train_df = pd.concat([train_df.reset_index(drop=True), train_y_df.reset_index(drop=True)], axis=1)
train_df

In [8]:
train_df[[0, 'y']].plot()

The data is clearly non-linear

## Lineral Regression

In [9]:
lr_model = LinearRegression()
history = lr_model.fit(X_train, y_train)
print(lr_model.score(X_test, y_test))

In [10]:
y_pred = lr_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mse

In [11]:
test_pred_df = pd.DataFrame(y_pred, columns=['y_pred'])
test_y_df = pd.DataFrame(y_test, columns=['y'])
test_df = pd.concat([test_pred_df.reset_index(drop=True), test_y_df.reset_index(drop=True)], axis=1)
test_df[['y', 'y_pred']].plot(figsize=(20, 6))

In [12]:
print(f'y = a * x + b = {lr_model.coef_} * x + {lr_model.intercept_}')

In [13]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
train_sizes = np.linspace(0.1, 1.0, 5)
train_sizes_abs, train_scores, test_scores = learning_curve(lr_model, X, y, cv=cv, train_sizes=train_sizes, scoring='neg_mean_squared_error')


In [14]:
print('train_sizes_abs', train_sizes_abs)
print('train_scores', train_scores)
print('test_scores', test_scores)

In [15]:
# Initialize plots
fig, axs = plt.subplots(1, 3, figsize=(18, 5))
metrics = ['r2', 'mape', 'mse']
scorings = ['r2', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error']

# Loop through each metric
for idx, (metric, scoring) in enumerate(zip(metrics, scorings)):
    # Get train and test scores
    train_sizes, train_scores, test_scores = learning_curve(lr_model, X, y, cv=cv, n_jobs=-1, train_sizes=train_sizes, scoring=scoring)

    # Calculate means and standard deviations for train and test scores
    train_scores_mean = np.mean(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    # Adjust scoring for display (if necessary)
    if scoring.startswith('neg_'):
        train_scores_mean = -train_scores_mean
        test_scores_mean = -test_scores_mean

    # Plot learning curves
    axs[idx].fill_between(train_sizes, train_scores_mean - train_scores_std,
                          train_scores_mean + train_scores_std, alpha=0.1, color="r")
    axs[idx].fill_between(train_sizes, test_scores_mean - test_scores_std,
                          test_scores_mean + test_scores_std, alpha=0.1, color="g")
    axs[idx].plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    axs[idx].plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")

    # Set plot titles and labels
    axs[idx].set_title(f'Learning Curve ({metric.upper()})')
    axs[idx].set_xlabel('Training examples')
    axs[idx].set_ylabel(metric.upper())
    axs[idx].legend(loc="best")

# Show plot
plt.tight_layout()
plt.show()


### Results interpretation

## Neural Networks

In [23]:
layers_options = [1, 5, 25]
optimizers_options = {'SGD': SGD, 'RMSProp': RMSprop, 'Adam': Adam}
learning_rates = [0.1, 0.01, 0.001]
loss_function = 'mean_squared_error'

# layers_options = [1, 5, 25]
# optimizers_options = {'SGD': SGD, 'RMSProp': RMSprop, 'Adam': Adam}
# learning_rates = [0.1]
# loss_function = 'mean_squared_error'

In [42]:
def r2_keras(y_true, y_pred):
    SS_res = tf.reduce_sum(tf.square(y_true - y_pred))
    SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return 1 - SS_res / (SS_tot + tf.keras.backend.epsilon())

In [50]:

model.compile(
    optimizer=SGD(learning_rate=0.1),
    loss='mean_squared_error',
    metrics=[r2_keras, 'mape', 'mse']
)


In [51]:

def plot_learning_curves(history, title):
    plt.figure(figsize=(10, 6))

    plt.plot(history.history['loss'], label='MSE (training loss)')
    plt.plot(history.history['mape'], label='MAPE')
    plt.plot(history.history['r2_keras'], label='R^2')

    plt.plot(history.history['val_loss'], label='MSE (val loss)')
    plt.plot(history.history['val_mape'], label='MAPE (val)')
    plt.plot(history.history['val_r2_keras'], label='R^2 (val)')

    plt.title(title)
    plt.ylabel('Loss / MAPE')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()


In [52]:
from keras import Input

model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1))
model.compile(
    optimizer=SGD(learning_rate=0.1),
    loss='mean_squared_error',
)

history = model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test, y_test)
)

print(history.history.keys())

plt.figure(figsize=(10, 6))
print(history.history['loss'])
plt.plot(history.history['loss'], label='MSE (training loss)')
plt.ylabel('Loss / MAPE')
plt.xlabel('Epoch')
plt.legend()
plt.show()


In [53]:
for layers in layers_options:
    for optimizer_name, Optimizer in optimizers_options.items():
        for lr in learning_rates:
          title = f"Layers: {layers}, Optimizer: {optimizer_name}, LR: {lr}"  #, R^2: {r_square:.2f}"
          print('*' * 100)
          print(title)
          model = define_model(layers, Optimizer, lr)
          history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))
          plot_learning_curves(history, title)